<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/Large-XLM/blob/master/fur_NMT_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Clone fastBPE and XLM

In [ ]:
%cd /content/
!git clone https://github.com/glample/fastBPE
%cd fastBPE
!g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast
%cd ..

In [ ]:
!git clone github.com/deterministic-algorithms-lab/Large-XLM
xlm_dir = '/content/drive/My Drive/Large-XLM'
%cd {xlm_dir}

In [ ]:
import os

## Apply BPE to Text Files

In [ ]:
test_data_root = '/content/test_data/'                                                                #txt files of test data must be directly under this folder
bpe_path = '/content/drive/XLM/checkpoints/Exp-1/codes_xnli_100'                                      #Path to file having bpe codes
fast_path = '/content/fastBPE/fast'                                                                   #Path to fast
out_path = '/content/bped/'                                                                           #Directory where the output files files of BPE are to be stored.
command = fast_path+' applybpe '
for f in os.listdir(root):
    in_path = os.path.join(root, f)
    final_command = command + os.path.join(out_path, f) + ' ' + in_path + ' ' + bpe_path
    print(final_command)
    os.system(final_command)

## Translate

In [ ]:
import re
checkpoint_path = '/content/drive/My\ Drive/XLM/checkpoints/Exp-1/fine-tune.pth'                   #Checkpoint for Model
output_path = '/content/drive/My\ Drive/XLM/evaluation/Exp-1/'                                     #Folder where translated files are to be stored.
bped_root = '/content/bped/'                                                                       #Directory Having BPE files
for f in os.listdir(bped_root):
    src_lang = f[-2]+f[-1]
    tgt_lang = [lang for lang in re.findall(r'..-..', f)[0].split('-') if lang!=src_lang][0]
    bped_filepath = os.path.join(bped_root, f)
    command = "cat "+bped_filepath+" | python translate.py --exp_name translate --src_lang "+src_lang\
                    +" --tgt_lang "+tgt_lang+" --model_path "+checkpoint_path+" --output_path "+os.path.join(output_path, 'hyp.'+f.replace('test.', ''))
    print(command)
    x=os.system(command)
    if x>>8!=0:
        break

## Join Together subwords

In [ ]:
import subprocess

def restore_segmentation(path):
    """
    Take a file segmented with BPE and restore it to its original segmentation.
    """
    assert os.path.isfile(path)
    restore_cmd = "sed -i -r 's/(@@ )|(@@ ?$)//g' %s"
    subprocess.Popen(restore_cmd % path, shell=True).wait()

In [ ]:
for f in os.listdir(output_path):
    restore_segmentation(os.path.join(output_path, f))